# Лабораторная работа на тему "Прямые методы решения слау"

### Вариант 10
Эскалаторный метод решения слау

##### описание метода


In [159]:
import numpy as np

In [160]:
def solve(A: np.ndarray, b: np.ndarray):
    n = A.shape[0]
    Ab = np.concatenate((A, np.expand_dims(-b, axis=1)), axis=1)
    Ab = np.concatenate((Ab, np.array([np.append(-b, 0)])), axis=0)
    
    Z = np.eye(n + 1)
    C = np.zeros((n+1, n))
    G = np.zeros((n+1, n))
    
    for k in range(n):
        # k - element of diag
        j = k

        for i in range(j, n+1):
            # C[i, j] = sum(Ab[i,s] * Z[s, j] for s in range(n))  # optimized variant because we stop at 1 and next go zeros
            C[i, j] = np.sum(Ab[i,:] * Z[:, j])

        for i in range(j, n+1):
            G[i, j] = C[i, j] / C[j, j]

        for i in range(k, -1, -1):
            Z[i, k+1] = -G[k+1,i] - np.sum(Z[i,(i+1):(k+1)] * G[k+1,(i+1):(k+1)])
    x = Z[:n, n]
    return x, Z, C, G



In [161]:
def check_input_matrix(A: np.ndarray, b: np.ndarray) -> bool:
    return np.all(A == A.T) and np.all(np.linalg.solve(A, b))

In [162]:
def test(A: np.ndarray, b: np.ndarray) -> None:
    print(f"исходная матрица A {A}")
    print(f"исходный вектор b {b}")
    print(f"проверка на валидность входных данных {check_input_matrix(A, b)}")

    my_x, Z, C, G = solve(A, b)
    np_x = np.linalg.solve(A, b)
    error = abs(my_x - np_x)

    print(
        f"мой x: {my_x}",
        f"точный x: {np_x}",
        f"Ошибка: {error}",
        f"Максимальная ошибка: {np.max(error):.6e}",
        sep="\n"
    )
    print(
        "-"*10,
        f"Z:{Z}",
        "-"*10,
        f"C:{C}",
        "-"*10,
        f"G:{G}",
        "-"*10,
        sep="\n"
    )

In [163]:
A = np.array([
    [2, 3],
    [3, 4]
])
b = np.array([5, 7])

In [164]:
test(A, b)

исходная матрица A [[2 3]
 [3 4]]
исходный вектор b [5 7]
проверка на валидность входных данных True
мой x: [1. 1.]
точный x: [1. 1.]
Ошибка: [1.77635684e-15 1.33226763e-15]
Максимальная ошибка: 1.776357e-15
----------
Z:[[ 1.  -1.5  1. ]
 [ 0.   1.   1. ]
 [ 0.   0.   1. ]]
----------
C:[[ 2.   0. ]
 [ 3.  -0.5]
 [-5.   0.5]]
----------
G:[[ 1.   0. ]
 [ 1.5  1. ]
 [-2.5 -1. ]]
----------


In [165]:
A = np.array([
    [12, 1.5, 2, 2.5, 1],
    [1.5, 10, 1.2, 1.8, 0.9],
    [2, 1.2, 14, 3, 1.5],
    [2.5, 1.8, 3, 16, 1.8],
    [1, 0.9, 1.5, 1.8, 15]
])
b = np.array([8.5, 6.3, 10.2, 12.8, 7.1])

In [166]:
test(A, b)

исходная матрица A [[12.   1.5  2.   2.5  1. ]
 [ 1.5 10.   1.2  1.8  0.9]
 [ 2.   1.2 14.   3.   1.5]
 [ 2.5  1.8  3.  16.   1.8]
 [ 1.   0.9  1.5  1.8 15. ]]
исходный вектор b [ 8.5  6.3 10.2 12.8  7.1]
проверка на валидность входных данных True
мой x: [0.43803631 0.37754672 0.47986729 0.56471356 0.30572575]
точный x: [0.43803631 0.37754672 0.47986729 0.56471356 0.30572575]
Ошибка: [1.11022302e-16 5.55111512e-17 5.55111512e-17 1.11022302e-16
 0.00000000e+00]
Максимальная ошибка: 1.110223e-16
----------
Z:[[ 1.         -0.125      -0.15456476 -0.16160959 -0.04551911  0.43803631]
 [ 0.          1.         -0.09681529 -0.13419502 -0.0587018   0.37754672]
 [ 0.          0.          1.         -0.1796962  -0.07755673  0.47986729]
 [ 0.          0.          0.          1.         -0.0842418   0.56471356]
 [ 0.          0.          0.          0.          1.          0.30572575]
 [ 0.          0.          0.          0.          0.          1.        ]]
----------
C:[[12.          0.       

In [167]:
def generate(size: int = 4, tryings: int = 10**7) -> tuple[np.ndarray, np.ndarray]:
    for _ in range(tryings):
        # A_random = np.random.randn(size, size)
        A_random = np.random.randint(-10, 10, size=(size, size))
        A = (A_random + A_random.T)

        b = np.random.randint(-10, 10, size=(size))
        if check_input_matrix(A, b):
            return A, b

In [168]:
A, b = generate(size=4)
A, b

(array([[-14,  -4,  -1,   6],
        [ -4,  14, -12,  -6],
        [ -1, -12,  -4,  11],
        [  6,  -6,  11, -12]]),
 array([-8,  6, -5,  4]))

In [169]:
test(A, b)

исходная матрица A [[-14  -4  -1   6]
 [ -4  14 -12  -6]
 [ -1 -12  -4  11]
 [  6  -6  11 -12]]
исходный вектор b [-8  6 -5  4]
проверка на валидность входных данных True
мой x: [ 0.37565024  0.06205103 -0.31855338 -0.468541  ]
точный x: [ 0.37565024  0.06205103 -0.31855338 -0.468541  ]
Ошибка: [0.00000000e+00 1.11022302e-16 5.55111512e-17 5.55111512e-17]
Максимальная ошибка: 1.110223e-16
----------
Z:[[ 1.         -0.28571429 -0.29245283  0.17937545  0.37565024]
 [ 0.          1.          0.77358491  0.78358751  0.06205103]
 [ 0.          0.          1.          0.35439361 -0.31855338]
 [ 0.          0.          0.          1.         -0.468541  ]
 [ 0.          0.          0.          0.          1.        ]]
----------
C:[[-14.           0.           0.           0.        ]
 [ -4.          15.14285714   0.           0.        ]
 [ -1.         -11.71428571 -12.99056604   0.        ]
 [  6.          -7.71428571   4.60377358 -11.72694263]
 [  8.          -8.28571429  -1.98113208  -5.4